# CDAW CME Catalog Creation Notebook
This notebook is for parsing the online version and creating a CDAW CME catalog we may use. Changes include: we'll calculate duration of events in C2, C3 detectors and write as CSV.

In [1]:
import requests
import pandas as pd

Need_To_Redownload = False

CDAW_Catalog_Filename = '../cdaw_catalog.txt'

All_CME_Catalog_Filename = '../all_cme_event_catalog.csv'  # this one is cleaned of weak/poor CME
CME_Catalog_Filename = '../cme_event_catalog.csv'  # this one is cleaned of weak/poor CME
Raw_CME_Catalog_Filename = '../raw_cme_event_catalog.csv' # this one is uncleaned and useful for calculation of non-CME time intervals

In [2]:
# pull the cme catalog to local disk
if Need_To_Redownload:
    
    cme_catalog_url = 'https://cdaw.gsfc.nasa.gov/CME_list/UNIVERSAL/text_ver/univ_all.txt'
    response = requests.get(cme_catalog_url, allow_redirects=True)
    
    # cache locally
    open(CDAW_Catalog_Filename, 'wb').write(response.content)

    # parse out data into string lines from byte block
    cdaw_raw_lines = response.content.decode('utf-8').split('\n')
    
    # download LASCO catalog now
    
    #lasco_hdr_url = 'https://lasco-www.nrl.navy.mil/lz/img_hdr.txt'   # level 0.5 data
    #lasco_hdr_url = 'https://lasco-www.nrl.navy.mil/lz/level_1/img_hdr.txt'
    #response = requests.get(lasco_hdr_url, allow_redirects=True)
    
    # cache locally
    # LASCO_Catalog_Filename = '../lasco_img_catalog_lvl_1.txt'
    # open(LASCO_Catalog_Filename, 'wb').write(response.content)

else:
    
    with open(CDAW_Catalog_Filename, 'r') as f:
        cdaw_raw_lines = f.readlines()
        
    print(f"Read %s CDAW lines" % len(cdaw_raw_lines))
        

Read 31632 CDAW lines


In [3]:
# parse remainder of lines into dataframe using regex pattern
# only date and time fields appear to be consistently same format,
# we have to guard against non-numbers in all of the other columns
def parse_data(cdaw_raw_lines:list)->pd.DataFrame:
    
    import re

    pattern_str = r"^(\d{4}/\d{2}/\d{2})\s+(\d{2}:\d{2}:\d{2})\s+(\S+)\s+(\S+)\s+(\S+)\s+" +\
                  r"(\S+)\s+(\S+)\s+(\S+)\s+(\S+?)\s+(\S+)\s+(\S+)\s+(\S+)\s+(\S[\S|\s]+)?$"
    pattern = re.compile(pattern_str)

    #snag field names from top of file. Its not entirely correct, 
    # but 'good enough' for now
    fields = cdaw_raw_lines[1].split()

    data_arr = { fld:[] for fld in fields }
    # parse remainder into columns
    for line in cdaw_raw_lines[4:]:
        m = pattern.match(line)
        if m:
            for grp in range(1,len(m.groups())+1): 
                #print(f"{grp}: %s" % m.group(grp))
                data_arr[fields[grp-1]].append(m.group(grp))
        else:
            print (f"Warning: cannot parse line:\n  {line}")

    raw_data = pd.DataFrame(data_arr)
    
    return raw_data

In [4]:
# Data Cleaning: filter out Bad/Poor data: Missing Linear speed ('------') and
# 'Very Poor Event' and 'None' in Remarks
#
def clean_data(data:pd.DataFrame)->pd.DataFrame:
    data = raw_data[raw_data['Linear'] != '----']
    #data = data[data.Remarks.str.contains('Very Poor Event') == False]
    data = data[data.Remarks.str.contains('None') == False]
    return data

In [5]:
# fix up data by getting rid of non-numeric chars in some columns, also
# explicitly cast as numbers
def fix_data_format (data:pd.DataFrame)->pd.DataFrame:
    data['width'] = pd.to_numeric(data['Width'])
    #data['central'] = pd.to_numeric(data['Central'])
    
    lspeed = []
    for val in data['Linear']:
        if val == '----': 
            val = 1.e-20

        lspeed.append(val)
    
    data['linear'] = pd.to_numeric(lspeed) # data['Linear'])
    data['mpa'] = pd.to_numeric(data['MPA'])
    #data['cpa_mpa'] = data['central'] - data['mpa']
    data['mass'] = pd.to_numeric([i.replace('*', '').replace('-','0') for i in data['Mass']])
    data['kinetic'] = pd.to_numeric([i.replace('*', '').replace('-','0') for i in data['Kinetic']])
    data['accel'] = pd.to_numeric([i.replace('*', '').replace('--','0') for i in data['Accel']])
    #data[['accel', 'Accel', 'mass', 'Mass','kinetic']]
    return data

### calculate durations for each detector (C2 1.5 - 6 Rsol and C3 is 3.7 - 30 Rsol)

#### T_c2_start == catalog time_start

Our formula for duration:
  700,000 * Det_max_Rsol /linear speed = T_duration_det

#### C2 Duration : 4200000 / speed = T_c2
 
C3 Start time delta (from C2) is C2_Start + (3.7 - 1.5) * R_sol
or
2.2 * R_sol  = T_c3_start_delta

#### C3 Delta Start is 1540000 + T_c2_start

C3 duration is (30 - 3.7) * R_sol / linear speed

#### C3 Duration: 18410000 / speed = T_c3

In [6]:
def add_durations(data:pd.DataFrame)->pd.DataFrame:
    c2_durations = [ 4200000./speed for speed in data['linear']]
    c3_durations = [ 18410000./speed for speed in data['linear']]
    # delta is C3 start time (est) after event start (at Sun).
    c3_start_delta = [1540000./speed for speed in data['linear']]
    
    # add durations to the catalog
    data['Duration_C2'] = c2_durations
    data['Duration_C3'] = c3_durations
    data['StartTime_C3_Delta'] = c3_start_delta
    
    return data

def fix_data(data:pd.DataFrame)->pd.DataFrame:
    
    data = fix_data_format(data)
    return add_durations(data)


## Run Pipeline

In [7]:
# run the pipeline
raw_data = parse_data(cdaw_raw_lines)
cleaned_data = clean_data(raw_data)
all_data = fix_data(cleaned_data)
all_uncleaned_data = fix_data(raw_data)
all_data[:1]

,Date,Time,Central,Width,Linear,2nd,order,speed,Accel,Mass,...,Remarks,width,linear,mpa,mass,kinetic,accel,Duration_C2,Duration_C3,StartTime_C3_Delta
0,1996/01/11,00:14:36,267,18,499,571,426,0,-64.3*,-------,...,Only C3\n,18,499,272,0.0,0.0,-64.3,8416.833667,36893.787575,3086.172345


In [8]:
len(all_data)

22051

In [13]:
# look at July 12th, 2000 -- should have CME at 02:49 + 
foo = all_uncleaned_data[all_uncleaned_data['Date']=='2000/07/12']
foo

,Date,Time,Central,Width,Linear,2nd,order,speed,Accel,Mass,...,Remarks,width,linear,mpa,mass,kinetic,accel,Duration_C2,Duration_C3,StartTime_C3_Delta
3242,2000/07/12,02:50:06,96,90,613,519,708,847,18.2,4.7e+15,...,None,90,6.130000e+02,90,4.700000e+15,8.800000e+30,18.2,6.851550e+03,3.003263e+04,2.512235e+03
3243,2000/07/12,05:26:05,268,47,445,396,498,597,8.3*,4.2e+14,...,None,47,4.450000e+02,276,4.200000e+14,4.200000e+29,8.3,9.438202e+03,4.137079e+04,3.460674e+03
3244,2000/07/12,11:06:05,20,144,1124,1285,958,0,-151.1*,-------,...,Very Poor Event; Only 3 points; Partial Halo\n,144,1.124000e+03,49,0.000000e+00,0.000000e+00,-151.1,3.736655e+03,1.637900e+04,1.370107e+03
3245,2000/07/12,11:50:05,49,56,287,245,329,381,3.5*,2.2e+14,...,None,56,2.870000e+02,46,2.200000e+14,9.100000e+28,3.5,1.463415e+04,6.414634e+04,5.365854e+03
3246,2000/07/12,16:50:05,282,96,475,447,506,503,2.2,1.0e+16,...,None,96,4.750000e+02,258,1.000000e+16,1.200000e+31,2.2,8.842105e+03,3.875789e+04,3.242105e+03
3247,2000/07/12,17:50:05,49,103,402,316,491,662,14.1*,1.0e+14,...,None,103,4.020000e+02,38,1.000000e+14,8.200000e+28,14.1,1.044776e+04,4.579602e+04,3.830846e+03
3248,2000/07/12,20:30:05,281,101,820,856,789,810,-3.2,7.4e+15,...,None,101,8.200000e+02,281,7.400000e+15,2.500000e+31,-3.2,5.121951e+03,2.245122e+04,1.878049e+03
3249,2000/07/12,23:06:07,72,33,----,----,----,----,------,4.4e+14*,...,Very Poor Event; Only C2; Unable to measure\n,33,1.000000e-20,78,4.400000e+14,0.000000e+00,0.0,4.200000e+26,1.841000e+27,1.540000e+26


In [12]:
# check for July 12 event in cleaned
foo = all_data[all_data['Date'] == '2000/07/12']
foo

,Date,Time,Central,Width,Linear,2nd,order,speed,Accel,Mass,...,Remarks,width,linear,mpa,mass,kinetic,accel,Duration_C2,Duration_C3,StartTime_C3_Delta
3244,2000/07/12,11:06:05,20,144,1124,1285,958,0,-151.1*,-------,...,Very Poor Event; Only 3 points; Partial Halo\n,144,1124,49,0.0,0.0,-151.1,3736.654804,16379.003559,1370.106762


## Write Data To disk

In [14]:
# This is both C2 and C3 detectors together, although they may not both observe the same CME
all_data.to_csv(All_CME_Catalog_Filename, index=True, index_label="CDAW_Id")
len(all_data)

22051

In [15]:
# This is both C2 and C3 detectors together, w/o filtering poor CME events out (needed
# for accurate calculation of CME non-times)
all_uncleaned_data.to_csv(All_CME_Catalog_Filename, index=True, index_label="CDAW_Id")
len(all_uncleaned_data)

31628

In [16]:
# Split into detector specific datasets
data = all_data
c2_data = data[data.Remarks.str.contains('Only C2')]
c3_data = data[data.Remarks.str.contains('Only C3')]
c2_c3_data = data[data.Remarks.str.contains('Only C2') == False]
c2_c3_data = c2_c3_data[c2_c3_data.Remarks.str.contains('Only C3') == False]
len(c2_data), len(c3_data), len(c2_c3_data)

(10591, 362, 11098)

In [17]:
c2_c3_data[:3]

,Date,Time,Central,Width,Linear,2nd,order,speed,Accel,Mass,...,Remarks,width,linear,mpa,mass,kinetic,accel,Duration_C2,Duration_C3,StartTime_C3_Delta
5,1996/01/31,06:52:13,274,47,158,219,99,0,-12.3*,2.5e+14*,...,Poor Event; Only 3 points\n,47,158,272,2.500000e+14,3.200000e+28,-12.3,26582.278481,116518.987342,9746.835443
8,1996/02/12,05:47:26,91,53,160,100,211,236,2.0*,1.3e+15,...,Poor Event\n,53,160,92,1.300000e+15,1.600000e+29,2.0,26250.000000,115062.500000,9625.000000
9,1996/02/17,02:06:31,279,73,317,257,378,532,9.9*,3.3e+14*,...,Very Poor Event\n,73,317,274,3.300000e+14,1.600000e+29,9.9,13249.211356,58075.709779,4858.044164


In [18]:
# persist dataset for CME where seen in both detectors
c2_c3_data.to_csv(CME_Catalog_Filename, index=True, index_label="CDAW_Id")

In [19]:
len(c2_c3_data)

11098

In [ ]:
#jul_cme = c2_c3_data[c2_c3_data["Date"] > '2000/07/01']
#jul_cme[:10]

In [ ]:
#mar_cme = c2_c3_data[c2_c3_data["Date"] > '2000/03/20']
#mar_cme[:3]